## SDK-PY Workshop

Today we are going to have a quick lesson on interacting with the DharitrI blockchain using Python.

We will learn to interact with the Network through a so-called network provider, and we will create, sign and broadcast transactions (including Smart Contract calls), query accounts state (including Smart Contract queries), and more.

First, let us meet the libraries we will be using for interacting with DharitrI. They are:

- [@terradharitri/sdk-py-core](https://github.com/TerraDharitri/drt-sdk-py-core)
- [@terradharitri/sdk-py-wallet](https://github.com/TerraDharitri/drt-sdk-py-wallet)
- [@terradharitri/sdk-py-network-providers](https://github.com/TerraDharitri/drt-sdk-py-network-providers)

Today we are going to:

- Create, sign and broadcast REWA and DCDT Transfers
- Query account information
- Interact with Smart Contracts

We'll do all these things using a Jupiter Notebook.

For our purposes, the application will interact with the Devnet. We are going to use the public DharitrI API as our entrypoint to the Network.

I'll use a wallet that contains some tokens so we can send transactions, but I'll show you how to create a new wallet, in case you don't have one already.

## Creating Wallets

### Generating the mnemonic

In [ ]:
from dharitri_sdk_wallet import Mnemonic

mnemonic = Mnemonic.generate()
words = mnemonic.get_words()

print(words)

: 

This is just an example, you **SHOULD NOT** display or share your mnemonic.

Given the mnemonic, we can generate keypairs:

In [ ]:
secret_key = mnemonic.derive_key(0)
public_key = secret_key.generate_public_key()

print("Secret key:", secret_key.hex())
print("Public key:", public_key.hex())

Using the keypair we can save it as a wallet. There are different wallet formats so we'll save it a PEM wallet. This is not recommended as it is not very safe, but it's good for testing as it is very easy to use.

In [ ]:
from pathlib import Path

from dharitri_sdk_core import Address
from dharitri_sdk_wallet import UserPEM

label = Address(public_key.buffer, "drt").bech32()
pem = UserPEM(label=label, secret_key=secret_key)
pem.save(Path("./output/wallet.pem"))

## REWA Transfers

### Fetching the nonce

We fetch the nonce using the network providers. We are using the DharitrI Api, so we are going to pass it the api url.

In [ ]:
from dharitri_sdk_network_providers import ApiNetworkProvider

api = ApiNetworkProvider("https://devnet-api.dharitri.org")
sender = Address.from_bech32("drt1seev64tevr927nm0hg7ux9zh94khcj0pxs3ckmnaswglfn0s5w4s4wjkpa")

nonce = api.get_account(sender).nonce

print(nonce)

### Crafting the move balance transaction

After fetching the nonce of the sender we can craft our transaction. Keep in mind that there's an ongoing effort to standardize the SDK's so they'll become consistent across all programming languages. That being said, how we craft the transaction might change in the future, but in this workshop we'll use the libraries as they are now.

In [ ]:
from dharitri_sdk_core import Address, TokenPayment, Transaction

receiver = Address.from_bech32("drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf")

tx = Transaction(
    nonce=nonce,
    sender=sender,
    receiver=receiver,
    value=TokenPayment.rewa_from_amount("1.0"),
    gas_limit=50000,
    chain_id="D"
)

print(tx.to_dictionary())

### Signing the transaction

We will crete a signer from the PEM file of our acount.

In [ ]:
from dharitri_sdk_wallet import UserSigner

signer = UserSigner.from_pem_file(Path("./output/workshop.pem"))

tx.signature = signer.sign(tx)
print("Signature", tx.signature.hex())

### Broadcasting the transaction

After our transaction is prepared and signed, it's ready to be broadcasted on the network.

In [ ]:
hash = api.send_transaction(tx)
print("Transaction hash:", hash)

## DCDT Transfers

For sending DCDT Tokens we should take the same steps as we did for transfering REWA. We fetch the account nonce, we create the transaction, we sign it and then we broadcast it.

When sending DCDT Tokens we set the value of the transaction to `0`. We also need to format the data field in a certain way. You can see how the data field should look [here](https://docs.dharitri.org/tokens/dcdt-tokens#transfers), but we don't need to worry about this since we have a builder that we'll be using and that will take care of formatting the field for us.

Let's fetch the nonce again. Because we've broadcasted the above transaction the nonce has incremented.

In [ ]:
nonce = api.get_account(sender).nonce

print(nonce)

Before instantiating the Transaction Builder we need to create a configuration object.

In [ ]:
from dharitri_sdk_core.transaction_builders import DefaultTransactionBuildersConfiguration

config = DefaultTransactionBuildersConfiguration(chain_id="D")

The **transaction builder** is parametrized at instantiation, and the transaction is obtained by invoking the `build()` method:

In [ ]:
from dharitri_sdk_core.transaction_builders import DCDTTransferBuilder

payment = TokenPayment.fungible_from_amount("WSTKN-b7792b", "100.00", 6)

builder = DCDTTransferBuilder(
    config=config,
    sender=sender,
    receiver=receiver,
    payment=payment,
    nonce=nonce,
    value=0
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

After the transaction is crafted it's time to sign it.

In [ ]:
tx.signature = signer.sign(tx)
print("Signature", tx.signature.hex())

The only step left now is to broadcast the transaction:

In [ ]:
hash = api.send_transaction(tx)
print("Transaction hash:", hash)

## Contract deployment and interactions

### Contract deployment

In order for us to interact with our smart contract it's necessary to deploy it onto the network. For the sake of our examples, we will proceed with deploying one ourselves.

Before deploying the contract make sure you have built the latest version.

For creating the deploy transaction we will use a `Contract Deployment Builder`.

In [ ]:
from dharitri_sdk_core import CodeMetadata
from dharitri_sdk_core.transaction_builders import ContractDeploymentBuilder

metadata = CodeMetadata(upgradeable=True, readable=True, payable=True, payable_by_contract=True)

builder = ContractDeploymentBuilder(
    config,
    owner=sender,
    deploy_arguments=[0],
    code_metadata=metadata,
    code=Path("./contracts/adder.wasm").read_bytes(),
    gas_limit=10000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

Of course you need to set the correct nonce and sign it and after that the transaction is ready to be broadcasted.

### Contract call

We can create a transaction that invokes a smart contract function:

In [ ]:
from dharitri_sdk_core.transaction_builders import ContractCallBuilder

contract_address = Address.from_bech32("drt1qqqqqqqqqqqqqpgq9ntwtuj3max5qlgmpvvuxknmaay5z9fdd8ssfxr0py")

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="add",
    caller=sender,
    call_arguments=[5],
    gas_limit=10000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

### Contract queries

For querying smart contracts we only need to use the network providers, we do not need to craft any transactions.

In [ ]:
from dharitri_sdk_core import ContractQueryBuilder
from dharitri_sdk_network_providers import ApiNetworkProvider

contract = Address.from_bech32("drt1qqqqqqqqqqqqqpgq9ntwtuj3max5qlgmpvvuxknmaay5z9fdd8ssfxr0py")

builder = ContractQueryBuilder(
    contract=contract,
    function="getSum",
    call_arguments=[],
    caller=sender
)

query = builder.build()

network_provider = ApiNetworkProvider("https://devnet-api.dharitri.org")
response = network_provider.query_contract(query)

print("Return code:", response.return_code)
print("Return data:", response.return_data)

### Contract upgrade

You can also upgrade your contract. Replace the old code with new code.

In [ ]:
from dharitri_sdk_core.transaction_builders import ContractUpgradeBuilder

contract_address = Address.from_bech32("drt1qqqqqqqqqqqqqpgq9ntwtuj3max5qlgmpvvuxknmaay5z9fdd8ssfxr0py")
metadata = CodeMetadata(upgradeable=True, readable=True, payable=True, payable_by_contract=True)

builder = ContractUpgradeBuilder(
    config,
    contract=contract_address,
    owner=sender,
    upgrade_arguments=[0],
    code_metadata=metadata,
    code=Path("./contracts/adder.wasm").read_bytes(),
    gas_limit=10000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

: 

Remember to set the correct nonce and sign the transactions if you want to broadcast them.

That's all for today!

We've reached the end of the tutorial.

We've learned to interact with the network using a network provider, to create, sign and broadcast transactions, to query account state, and to deploy and interact with smart contracts.

Thank you for listening!

Don't forget to check out our official documentation [here](https://docs.dharitri.org) and the workshops repository.